In [2]:

import pandas as pd
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
import os
from langchain.chains import LLMChain
import numpy as np

llm = OpenAI()

In [5]:
from langchain.document_loaders import UnstructuredExcelLoader
loader = UnstructuredExcelLoader("./data/ABSA.xlsx", mode="elements")
docs = loader.load()

In [6]:
docs[1].metadata['page_name']

'Aspects clean for Upwork specia'

In [7]:
df = pd.read_excel('./data/ABSA.xlsx', sheet_name=docs[1].metadata['page_name'])  # sheet_name is optional
df.columns

Index(['Unnamed: 0', 'Effectiveness', 'Price', 'Taste', 'Side effects',
       'Quality control', 'Varied results', 'Ingredients', 'date', 'name',
       'title', 'content', 'rating'],
      dtype='object')

In [8]:
review_df = df.loc[:,['content' ]]
review_df.head()

,content
0,I have been taking these gummies (along with t...
1,"They are a little expensive, but they seem to ..."
2,For so long I never bought these because I did...
3,Mi comentario positivo no es por que me hayan ...
4,I have tried this product a few times over the...


In [9]:
review_df['sentiment'] = np.nan

In [10]:
review_df.head()

,content,sentiment
0,I have been taking these gummies (along with t...,NaN
1,"They are a little expensive, but they seem to ...",NaN
2,For so long I never bought these because I did...,NaN
3,Mi comentario positivo no es por que me hayan ...,NaN
4,I have tried this product a few times over the...,NaN


In [11]:
review_df.loc[0, 'sentiment'] = 'positive'

In [12]:
review_df = review_df.iloc[:10,:]


In [13]:
prompt_template = "You is an expert at NLP and sentiment analysis. \
    There is a review from an Amazon customer. The content of the review is:{content} \
    Please perform global sentiment analysis. Your output should be: positive, neutral or negative. \
    If you don't know, just say 'I don't know and don't output'\
    sentiment output:"
prompt = PromptTemplate.from_template(prompt_template)
chain = LLMChain(llm=llm, prompt=prompt)

for idx, row in review_df.iterrows():    
    content = row['content']
    result = chain.run(content=content)
    review_df.loc[idx, 'sentiment'] = result

    
    

In [14]:
review_df.to_csv('sa.csv')